Import required packages

In [ ]:
import os
import json
import csv
import xml.etree.ElementTree as ET
import urllib
import urllib.request
import shutil
import re
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
import http.cookiejar
from bs4 import BeautifulSoup
!pip install elsapy
from elsapy.elsclient import ElsClient
from elsapy.elsprofile import ElsAuthor, ElsAffil
from elsapy.elsdoc import FullDoc, AbsDoc
!pip install scidownl
!pip install slate3k
import slate3k as slate
from scidownl.scihub import *
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

This is a folder in google drive which contains the citations list file

In [75]:
from google.colab import drive
drive.mount('/ProjectNLP', force_remount=True)

Mounted at /ProjectNLP


creating this directory to store the downloaded .xml files

In [76]:
!mkdir  '/content/NLP'

mkdir: cannot create directory ‘/content/NLP’: File exists


changing the directory to the path that contains the citation list file

In [77]:
os.chdir('/ProjectNLP') 

The task was to get the urls of the articles which have an open access.

Steps followed:

Go to the following link:

https://www.sciencedirect.com/search/advanced?qs=highlights&articleTypes=FLA&accessTypes=openaccess&show=100&offset=3600 


*   This link is obtained by filtering out the documents with "open access" and keyword "highlights"

*   Click on Export and select Export citations to text

*   We get a list of urls







In [78]:
def extractURLs(fileContent):
     urls = re.findall('\(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', fileContent.lower())

    #  for url in urls:
    #         print(url)
     return urls

Getting the urls from the citation files and storing it in variable

In [79]:
# finalurl=[]
# for myFile in os.listdir(os.getcwd()+"/My Drive/ProjectNLP"):
#   print(myFile)
#   myFile = open(os.getcwd()+"/My Drive/ProjectNLP/"+ myFile,encoding="utf8")
#   fileContent = myFile.read()
#   finalurl.append(extractURLs(fileContent))
myFile = open(os.getcwd()+"/My Drive/ProjectNLP/"+ "aipubsumu.txt",encoding="utf8")
fileContent = myFile.read()
listURL=(extractURLs(fileContent))

In [80]:
os.chdir('/content')

Pass the API key generated from Elsevier portal

In [9]:
key = '1cdd52bb8881a480836964f7775f0620'
client = ElsClient(key)

We pass the extracted URLs which are stored in the list and download the scientific articles in .xml format

In [82]:
listPII=[]

In [83]:
for url in listURL:
    if url.startswith("(http://www.sciencedirect.com"):
        pii = url.replace("(http://www.sciencedirect.com/science/article/pii/", "").replace(")", "").replace("\n", "")
        if not pii in listPII:
            listPII.append(pii)
print(listPII)

['s0957417413009615', 's2590188519300162', 's0957417418306535', 's0888613x13002041', 's0957417419302489', 's107158191630074x', 's0888613x17302335', 's1071581916000021', 's0925231219301961', 's0957417417300751', 's0963868717301798', 's0736584518303831', 's0921889016306285', 's0957417419301903', 's0888613x1400067x', 's0888613x14001133', 's259018851930006x', 's0888613x1300234x', 's0888613x14000607', 's1071581919300552', 's0957417419301812', 's0020025519304864', 's0142061518304319', 's0933365716302950', 's0306437915000459', 's0921889015003000', 's1071581916300866', 's1071581918300016', 's1071581916301380', 's0957417415003590', 's0921889018307474', 's0957417417307698', 's1071581917301404', 's0921889014002164', 's1071581916300453', 's0952197619301253', 's1071581918300971', 's0306437919304909', 's0957417414006472', 's0925231217309864', 's0736584515000666', 's1071581917300988', 's0888613x14000206', 's0950705118301394', 's1875952116300040', 's0888613x14000851', 's0740818818301828', 's2590188519

In [84]:
len(listPII)

66

In [85]:
!mkdir '/Test_data'

mkdir: cannot create directory ‘/Test_data’: File exists


In [86]:
os.chdir('/Test_data')

Method to split the text into sentences

In [87]:
import re


def getSentences(text):
  sentences=[]
  intersent=''
  for s in re.split('[.][\s]([A-Z])',text):
    # print(s.split(" "))
    if len(s)==1:
      intersent=s
    else:
      if len(re.split('[\s]',intersent+s))>4:  
        if not '\n' in  intersent+s:     
          sentences.append(intersent+s)
        intersent=''
  return sentences  


In [88]:
def getSentencesHighlights(text):
  sentences=[]
  intersent=''
  for s in re.split('[•]([A-Z])',text):
    # print(s.split(" "))
    if len(s)==1:
      intersent=s
    else:
      if len(re.split('[\s]',intersent+s))>4:
        sentences.append(intersent+s)
        intersent=''
   
  return sentences

Method to get the "Highlights" section from the papers


In [89]:
def getHighlights(tree):
  root = tree.getroot()
  highlightswithoutpreprocess = ""
  for tag in root.iter():
    if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}abstract-sec'):
      for t in tag:
        if (t.tag == '{http://www.elsevier.com/xml/common/dtd}simple-para'):
          for tg in t:
            if (tg.tag == '{http://www.elsevier.com/xml/common/dtd}list'):
              for g in tg:
                if (g.tag == '{http://www.elsevier.com/xml/common/dtd}list-item'):
                  for ti in g:
                    if (ti.tag == '{http://www.elsevier.com/xml/common/dtd}para'):
                      highlightswithoutpreprocess += str(ti.text)
                      highlightswithoutpreprocess += " "
  return highlightswithoutpreprocess


Method to get the "Introduction" section from the papers

In [90]:
# sentence_index=[]
def getIntro(tree):
  root = tree.getroot()
  introText=""
  isIntroFound = False
  for child in root.iter():
    if child.tag == '{http://www.elsevier.com/xml/common/dtd}section' and isIntroFound == False:
      for introtag in child:
        introText+=" "+introtag.text
        for subintroTag in introtag:
          introText+=" "+subintroTag.tail
      isIntroFound = True 
  return introText
# for s in introText.split(sep='.'):
#   if(len(s)>50):
#     sentence_index.append(s)

Method to get the "Abstract" section from the papers

In [91]:
def getAbstract(tree):
  root = tree.getroot()
  # sentence_index=[]
  abstract_text = ""
  for tag in tree.iter():
    if(tag.tag == '{http://purl.org/dc/elements/1.1/}description'):
      abstract_text+= str(tag.text)
  return abstract_text
  # abstract_df[filename] = abstract_text  
# for s in abstract_text.split (sep = '.'):
#   if (len(s)>50):
#     sentence_index.append(s)


Method to get the "All other" section from the paper


In [92]:
def getOC(tree):
  # root = tree.getroot()
  # oc_text = ""
  # # sentence_index=[]
  # for tag in tree.iter():
  #   if (tag.tag in notimptags):
  #     continue
  #   else:
  #     if (tag.text!= None):
  #       oc_text+= str(tag.text)
  # return oc_text
  root = tree.getroot()
  introText=""
  isIntroFound = False
  for child in root.iter():
    if child.tag == '{http://www.elsevier.com/xml/common/dtd}section' and isIntroFound == False:
      for introtag in child:
        introText+=" "+introtag.text
        for subintroTag in introtag:
          introText+=" "+subintroTag.tail           
  return introText
      # other_content_df[filename] = oc_text
# for s in oc_text.split(sep = '.'):
#   if (len(s)>50) and not s.startswith('\n') and s[1].isupper():
#     sentence_index.append(s)

Method to get the keywords from the paper

In [93]:
def getKeywords(tree):
  keyword=[]
  # for tag in tree.iter():
  #   if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}bold'): #get bold words
  #     keyword.append(tag.text)
        # bold_words[filename] = (bold)
  # for tag in tree.iter():
  #   if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}italic'): #get italic words
  #     keyword.append(tag.text)
        # italic_words[filename] = (italics)
  for tag in tree.iter():
    if (tag.tag == '{http://www.elsevier.com/xml/common/dtd}keyword'):  # get keywords
      for t in tag:
        if (t.tag == '{http://www.elsevier.com/xml/common/dtd}text'):
          keyword.append(t.text)
                # keyword_main[filename] = (keywords)
  # for tag in tree.iter():
  #   if(tag.tag == '{http://purl.org/dc/elements/1.1/}title'): # title
  #     keyword.append(tag.text)
  return keyword
        # title_words[filename] = (title)

In [94]:
Abstract={}
Highlights={}
OtherContent={}
OtherContent_Dict={}
Abstract_Dict={}
Highlights_Dict={}
Intro={}
Intro_Dict={}
keywordmain={}
for pii_id in listPII:
  try:
    pag=urllib.request.urlopen( "https://api.elsevier.com/content/article/pii/"+pii_id+"?apiKey=1cdd52bb8881a480836964f7775f0620")
    tree = ET.parse(pag)    
    Abstract_Dict[pii_id]=getAbstract(tree)
    Abstract[pii_id]=getSentences(getAbstract(tree))
    Highlights_Dict[pii_id]=getHighlights(tree)
    Highlights[pii_id]=getSentences(getHighlights(tree))
    Intro_Dict[pii_id]=getIntro(tree)
    Intro[pii_id]=getSentences(getIntro(tree))
    OtherContent_Dict[pii_id]=getOC(tree)
    OtherContent[pii_id]=getSentences(getOC(tree))
    keywordmain[pii_id]=getKeywords(tree)
  except:
    print("Failed for "+pii_id)


In [95]:
print(Abstract_Dict)
print(Highlights_Dict)
print(Intro_Dict)
print(keywordmain)

{'s0957417413009615': '\n                  This study improves the recognition accuracy and execution time of facial expression recognition system. Various techniques were utilized to achieve this. The face detection component is implemented by the adoption of Viola–Jones descriptor. The detected face is down-sampled by Bessel transform to reduce the feature extraction space to improve processing time then. Gabor feature extraction techniques were employed to extract thousands of facial features which represent various facial deformation patterns. An AdaBoost-based hypothesis is formulated to select a few hundreds of the numerous extracted features to speed up classification. The selected features were fed into a well designed 3-layer neural network classifier that is trained by a back-propagation algorithm. The system is trained and tested with datasets from JAFFE and Yale facial expression databases. An average recognition rate of 96.83% and 92.22% are registered in JAFFE and Yale da

Method for tokenisation, removing stopwords and lemmatization

In [ ]:
def getValidTokens(sent):
  stopWords = set(stopwords.words('english'))
  tokenizer = RegexpTokenizer(r'\w+')
  words = tokenizer.tokenize(sent)
  wordsFiltered = []

  for w in words:
      w = str(w).lower()
      if w not in stopWords:
          wordsFiltered.append((w))
  wordnet_lemmatizer = WordNetLemmatizer()
  lemmatized_words = [wordnet_lemmatizer.lemmatize(token, pos="v") for token in wordsFiltered]
  return lemmatized_words

Downloading the papers as the pdf files and separting the content of paper into "Abstract", "Highlights", "Introduction" and "All other" section


In [72]:
def getAbstractHighlightsKeywords(pii_id):
  keywordsres=[]
  xml_url='https://www.sciencedirect.com/science/article/abs/pii/'+pii_id


  site= "http://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/getHistoricalData.jsp?symbol=JPASSOCIAT&fromDate=1-JAN-2012&toDate=1-AUG-2012&datePeriod=unselected&hiddDwnld=true"
  hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

  req = urllib.request.Request(xml_url, headers=hdr)

  try:
      page = urllib.request.urlopen(req)
  except (urllib.HTTPError, e):
      print (e.fp.read())

  content = page.read()
  soup = BeautifulSoup(content, 'html.parser')
  keywords=soup.findAll("div", {"class": "keyword"})
  for keyword in keywords:
    keywordsres.append(keyword.get_text())
  results = soup.find(id='abstracts')
  
  strR=''
  for res in results:
    strR+=(res.getText())
  
  abstract=strR.split('Abstract')[1]
  Highlights=strR.split('Abstract')[0]
  return abstract,Highlights.split('Highlights')[1],keywordsres

In [ ]:
Abstract={}
Highlights={}
OtherContent={}
OtherContent_Dict={}
Abstract_Dict={}
Highlights_Dict={}
Intro={}
Intro_Dict={}
keywordmain={}
for pii_id in listPII:
  pii_doc =FullDoc(sd_pii=pii_id)
  docid=''
  if pii_doc.read(client):
      print ("pii_doc.title: ", pii_doc.id)
      docid=pii_doc.id
      pii_doc.write()
      abstract,highlight,keywords=getAbstractHighlightsKeywords(pii_id)
      out = 'paper'
      try:
        sci = SciHub(docid[4:len(docid)], out).download(choose_scihub_url_index=1)
        paperName=(pii_doc.title[0:len(pii_doc.title)-1])
        punc=['/','?',':']
        for p in punc:
          if p in paperName:
            paperName=paperName.replace(p," ")
        paperName=paperName.replace("'","’")
      
        with open('paper/'+paperName+'.pdf','rb') as f:
          extracted_text = slate.PDF(f)
          text = [blk.replace("\n", " ") for blk in extracted_text]
          resText="\r\n".join(text)
        if len(resText.split('Introduction'))==2:
          contText=(resText.split('Introduction')[1]).split('References')[0]
        else:
          contText=''
          for t in range(len(resText.split('Introduction'))):
            if t!=0:
              contText+=resText.split('Introduction')[t]
        intro=getIntroDictCont(contText)
        if intro=='':
          raise Exception
        else:
          Intro[pii_id]=getSentences(intro)
          Intro_Dict[pii_id]=intro

        OtherContent[pii_id]=getSentences(contText)
        OtherContent_Dict[pii_id]=contText
        
        Abstract_Dict[pii_id]=(abstract)
        Highlights_Dict[pii_id]=(highlight)
        Abstract[pii_id]=getSentences(abstract)
        Highlights[pii_id]=getSentences(highlight)
        keywordmain[pii_id]=keywords
      except:
        print("Failed to read:"+pii_doc.title[0:len(pii_doc.title)-1]+"with pii_id="+pii_id)


  else:
      print ("Read document failed.")